In [4]:
# Gerekli kütüphaneleri yükleme
!pip install huggingface_hub torch pillow contexttimer decord diffusers==0.30.1 accelerate==0.30.0 torch==2.1.2 torchvision==0.16.2 numpy==1.24.4 einops ftfy ipython opencv-python-headless==4.10.0.84 imageio==2.33.1 imageio-ffmpeg==0.5.1 packaging pandas plotly pre-commit pycocoevalcap pycocotools python-magic scikit-image sentencepiece spacy streamlit timm==0.6.12 tqdm transformers==4.39.3 wheel torchmetrics tiktoken jsonlines tensorboardX


In [6]:
# GitHub reposunu klonlama
!git clone https://github.com/jy0205/Pyramid-Flow.git
%cd Pyramid-Flow

fatal: destination path 'Pyramid-Flow' already exists and is not an empty directory.
/content/Pyramid-Flow


In [7]:
# Hugging Face'den modeli indirme
from huggingface_hub import snapshot_download

model_path = './pyramid-flow-model'
!mkdir -p {model_path}
snapshot_download("rain1011/pyramid-flow-sd3", local_dir=model_path, local_dir_use_symlinks=False, repo_type='model')


KeyboardInterrupt: 

In [14]:
# Gerekli modülleri içe aktarma
import torch
from PIL import Image
from pyramid_dit import PyramidDiTForVideoGeneration
from diffusers.utils import load_image, export_to_video
# Görüntü yükleme için gerekli kütüphane
from google.colab import files

In [9]:
# GPU ayarları
torch.cuda.set_device(0)
model_dtype, torch_dtype = 'bf16', torch.bfloat16

In [10]:
# Modeli yükleme
model = PyramidDiTForVideoGeneration(
    model_path,
    model_dtype,
    model_variant='diffusion_transformer_768p',
)

model.vae.to("cuda")
model.dit.to("cuda")
model.text_encoder.to("cuda")
model.vae.enable_tiling()

An error occurred while trying to fetch ./pyramid-flow-model/diffusion_transformer_768p: Error no file named diffusion_pytorch_model.safetensors found in directory ./pyramid-flow-model/diffusion_transformer_768p.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


using half precision
Using temporal causal attention
We interp the position embedding of condition latents


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

An error occurred while trying to fetch ./pyramid-flow-model/causal_video_vae: Error no file named diffusion_pytorch_model.safetensors found in directory ./pyramid-flow-model/causal_video_vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


The latent dimmension channes is 16
The start sigmas and end sigmas of each stage is Start: {0: 1.0, 1: 0.8002399489209289, 2: 0.5007496155411024}, End: {0: 0.6669999957084656, 1: 0.33399999141693115, 2: 0.0}, Ori_start: {0: 1.0, 1: 0.6669999957084656, 2: 0.33399999141693115}


In [11]:
# Metin-video oluşturma örneği
def generate_text_to_video(prompt, output_path):
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=True, dtype=torch_dtype):
        frames = model.generate(
            prompt=prompt,
            num_inference_steps=[20, 20, 20],
            video_num_inference_steps=[10, 10, 10],
            height=768,
            width=1280,
            temp=16,
            guidance_scale=9.0,
            video_guidance_scale=5.0,
            output_type="pil",
            save_memory=True,
        )

    export_to_video(frames, output_path, fps=24)
    return output_path


In [12]:
# Görüntü-video oluşturma örneği
def generate_image_to_video(image_path, prompt, output_path):
    image = Image.open(image_path).convert("RGB").resize((1280, 768))

    with torch.no_grad(), torch.cuda.amp.autocast(enabled=True, dtype=torch_dtype):
        frames = model.generate_i2v(
            prompt=prompt,
            input_image=image,
            num_inference_steps=[10, 10, 10],
            temp=16,
            video_guidance_scale=4.0,
            output_type="pil",
            save_memory=True,
        )

    export_to_video(frames, output_path, fps=24)
    return output_path

In [13]:
# Kullanım örnekleri
text_prompt = "A movie trailer featuring the adventures of the 30 year old space man wearing a red wool knitted motorcycle helmet, blue sky, salt desert, cinematic style, shot on 35mm film, vivid colors"
text_to_video_output = generate_text_to_video(text_prompt, "./text_to_video_sample.mp4")
print(f"Text-to-video örneği oluşturuldu: {text_to_video_output}")


100%|██████████| 16/16 [05:32<00:00, 20.79s/it]


Text-to-video örneği oluşturuldu: ./text_to_video_sample.mp4


In [18]:
# Görüntüyü yükleme
uploaded = files.upload()

image_path = next(iter(uploaded))

Saving the_great_wall.jpg to the_great_wall.jpg


In [19]:
# Image-to-video oluşturma
image_prompt = "FPV flying over the Great Wall"
image_to_video_output = generate_image_to_video(image_path, image_prompt, "./image_to_video_sample.mp4")
print(f"Image-to-video örneği oluşturuldu: {image_to_video_output}")


100%|██████████| 16/16 [05:47<00:00, 21.74s/it]


Image-to-video örneği oluşturuldu: ./image_to_video_sample.mp4
